# Tool calling

Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs. This can ve useful for task that require access to external data or services.

## Load the .env variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the enviroment variables")

## Functions as a tool for the agent

In [2]:
# import Agent, Runner and **function_tool**, then we define a llm model
from agents import Agent, Runner, function_tool, WebSearchTool

from pydantic import BaseModel

llm_model = "gpt-4o-mini"

In [9]:
@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return f"sunny"

@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting temperature for {city}")
    return f"70 degrees"

weather_agent = Agent(
    name = "Weather Agent",
    instructions = """
        You are the local weather agent. 
        You are given a city and you need to tell the weather and temperature.
        For any unrelated queries, say i can't help with that./
    """,
    model = llm_model,
    tools = [get_weather, get_temperature]
)

result = await Runner.run(weather_agent, "What is the weather for Dallas?")
result.final_output

Getting weather for Dallas
Getting temperature for Dallas


'The weather in Dallas is sunny, and the temperature is 70 degrees.'

## OpenAI agent SDK built-in tools

In [10]:
from agents import WebSearchTool

# create a news agent
news_agent = Agent(
    name = "News Reporter",
    instructions = """
        You are a news reporter. 
        Your jobs is to find recent news articles on the internet about US politics.
    """,
    model = llm_model,
    tools = [WebSearchTool()]
)

result = await Runner.run(news_agent, "find news")
print(result.final_output)

Here are some recent developments in U.S. politics as of October 14, 2025:

**Massive Protests Planned Against President Trump**

Millions of Americans are expected to participate in the second "No Kings" day protest on October 18, 2025, in over 2,500 locations across the U.S. and internationally. Organized by Indivisible and a coalition of labor unions and activist groups, the demonstrations aim to oppose what they describe as President Donald Trump's authoritarianism and expansion of executive power during his second term. The movement, which first mobilized in June with over five million participants, criticizes Trump’s actions, such as deploying federal forces, censoring dissent, and suggesting a third presidential term. ([time.com](https://time.com/7325683/no-kings-protests-oct-18/?utm_source=openai))

**Presidential Medal of Freedom Awarded Posthumously to Charlie Kirk**

On October 14, 2025, President Donald Trump posthumously awarded conservative activist Charlie Kirk the Presi

## Example of an agent with web search tool

In [4]:
# run script news_agent.py